## Testes de valência para o projeto final de IA369Y 2 Semestre 2018

Passos para tratar os dados com valência, testar e escolher um classificador para utilizar no projeto final de IA369Y.

1) Remover espaços duplos, quebras de linha, números e links do dataset e das frases a serem testadas.

2) Remover stopwords e aplicar o stemmer.

3) Treinar os classificadores.

4) Realizar as predições com os classificadores.

5) Avaliar as medidas obtidas com os classificadores.

In [1]:
import sys
sys.path.append('../..')

import csv
import codecs
import copy
import re
from random import shuffle

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from nltk import word_tokenize
import gensim

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import Normalizer, MinMaxScaler
from sklearn.decomposition import TruncatedSVD, LatentDirichletAllocation
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, GaussianNB, ComplementNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix
from sklearn.externals import joblib

from utils import tokenizer, load_six_emotions, load_3_emotions

%matplotlib inline

np.warnings.filterwarnings('ignore')
np.random.seed(12345)

def highlight_max(data, color='green'):
    attr = f'background-color: {color}; color: white; font-weight: bold;'
    #remove % and cast to float
    data = data.replace('%','', regex=True).astype(float)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_max = data == data.max()
        return [attr if v else '' for v in is_max]
    else:  # from .apply(axis=None)
        is_max = data == data.max().max()
        return pd.DataFrame(np.where(is_max, attr, ''),
                            index=data.index, columns=data.columns)

### Matriz de Resultados

In [2]:
classf = {
    'MultinomialNB': 0,
    'ComplementNB': 0,
    'LogisticRegression': 0,
    'RandomForestClassifier': 0,
    'KNeighborsClassifier': 0,
    'MLPClassifier': 0,
    'LinearSVC': 0,
    'SVC': 0
}

matriz_resultados = {
    'all': {
        'tfidf': copy.deepcopy(classf),
        'tfidf+lsa': copy.deepcopy(classf),
        'tfidf+lda': copy.deepcopy(classf),
        'count': copy.deepcopy(classf),
        'count+lsa': copy.deepcopy(classf),
        'count+lda': copy.deepcopy(classf),
        'tfidf+count+w2c': copy.deepcopy(classf),
    },
    'tweets': {
        'tfidf': copy.deepcopy(classf),
        'tfidf+lsa': copy.deepcopy(classf),
        'tfidf+lda': copy.deepcopy(classf),
        'count': copy.deepcopy(classf),
        'count+lsa': copy.deepcopy(classf),
        'count+lda': copy.deepcopy(classf),
        'tfidf+count+w2c': copy.deepcopy(classf),       
    },
    'titulo_noticias': {
        'tfidf': copy.deepcopy(classf),
        'tfidf+lsa': copy.deepcopy(classf),
        'tfidf+lda': copy.deepcopy(classf),
        'count': copy.deepcopy(classf),
        'count+lsa': copy.deepcopy(classf),
        'count+lda': copy.deepcopy(classf),
        'tfidf+count+w2c': copy.deepcopy(classf),
    }
}

## Datasets

Para validar, serão utilizados dois datasets.

Os dos datasets foram obtidos do site minerando dados.

O primeiro deles tem tweets de política de Minas Gerais com rótulos de valência: positivo, negativo e neutro. Foi feito um tratamento para eliminar tweets repetidos e dessa forma sobraram 3016 tweets.

O segundo contém 2123 títulos de notícias com rótulos de valência: positivo, negativo e neutro.

In [3]:
path = '/home/rdenadai/vagrant/python-dev/sentiment-analysis-2018-president-election/dataset/valencia/'

#Carregando os datasets
def carregar(filename):
    frases = []
    with open(filename, 'r', encoding='utf-8') as h:
        reader = csv.reader(h, delimiter='|')
        for row in reader:
            frase = tokenizer(row[0]).strip()
            valencia = row[1].upper()
            if len(frase) > 5:
                frases.append((valencia, frase))
    return frases

In [4]:
#Carrega os datasets em separado
tweets_mg = carregar(f'{path}tweets_mg_tratados.csv')
shuffle(tweets_mg)
titulo_noticias = carregar(f'{path}titulo_noticias.txt')
shuffle(titulo_noticias)

frases = tweets_mg + titulo_noticias
shuffle(frases)

print(frases[:5])
print('-' * 20)
print(tweets_mg[:5])
print('-' * 20)
print(titulo_noticias[:5])

[('POSITIVO', 'tip gent govern'), ('NEGATIVO', 'artig critic influênc indústr álcol brasil durant cop'), ('POSITIVO', 'mineir continu ser min estar nã quer mor outr estar espírit'), ('POSITIVO', 'consum energ país dezembr sub bas anual geraçã cair estar min'), ('POSITIVO', 'pmg autor roub pres políc milit bairr caban marian')]
--------------------
[('NEUTRO', 'govern expuls servidor feder prátic ilícit politic estar min'), ('NEUTRO', 'o abus aun estar mau sól son min sord tard tempran estalan'), ('NEGATIVO', 'inacredit govern min ger compr mais dois helicópter'), ('POSITIVO', 'timbet glob prevençã febr amarel reforc após mort macac prat'), ('NEUTRO', 'brazil índi pataxós form medicin ufmg ger estar min')]
--------------------
[('POSITIVO', 'info divulg empres mais inov brasil'), ('NEUTRO', 'crédit veícul cair junh segund cetip'), ('NEUTRO', 'itaú prev reversã expansion fiscal'), ('NEGATIVO', 'volátil ibovesp volt cair petrobr val'), ('POSITIVO', 'novidad merc precis sab')]


In [5]:
#all_datasets
afrases = []
avalencias = []
for valencia, frase in frases:
    afrases.append(frase)
    avalencias.append(valencia)
    
print(afrases[:5])
print(avalencias[:5])
print('-' * 20)

#tweets_mg
atweets_mg = []
aval_tweets_mg = []
for valencia, frase in tweets_mg:
    atweets_mg.append(frase)
    aval_tweets_mg.append(valencia)

print(atweets_mg[:5])
print(aval_tweets_mg[:5])
print('-' * 20)

#titulo_noticias
atitulo_noticias = []
aval_titulo_noticias = []
for valencia, frase in titulo_noticias:
    atitulo_noticias.append(frase)
    aval_titulo_noticias.append(valencia)

print(atitulo_noticias[:5])
print(aval_titulo_noticias[:5])

['tip gent govern', 'artig critic influênc indústr álcol brasil durant cop', 'mineir continu ser min estar nã quer mor outr estar espírit', 'consum energ país dezembr sub bas anual geraçã cair estar min', 'pmg autor roub pres políc milit bairr caban marian']
['POSITIVO', 'NEGATIVO', 'POSITIVO', 'POSITIVO', 'POSITIVO']
--------------------
['govern expuls servidor feder prátic ilícit politic estar min', 'o abus aun estar mau sól son min sord tard tempran estalan', 'inacredit govern min ger compr mais dois helicópter', 'timbet glob prevençã febr amarel reforc após mort macac prat', 'brazil índi pataxós form medicin ufmg ger estar min']
['NEUTRO', 'NEUTRO', 'NEGATIVO', 'POSITIVO', 'NEUTRO']
--------------------
['info divulg empres mais inov brasil', 'crédit veícul cair junh segund cetip', 'itaú prev reversã expansion fiscal', 'volátil ibovesp volt cair petrobr val', 'novidad merc precis sab']
['POSITIVO', 'NEUTRO', 'NEUTRO', 'NEGATIVO', 'POSITIVO']


In [6]:
def run_ml_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    accuracy = np.round(model.score(X_test, y_test) * 100, 2)
    print(f'Modelo   : {model.__class__.__name__}')
    print(f'Acurácia : {accuracy}%')
    print('-' * 20)
    return accuracy

def split_data(X, y):
    test_size = .33
    random_state = 0
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    return {
        'X_train': X_train,
        'y_train': y_train,
        'X_test': X_test,
        'y_test': y_test
    }

## Classificadores

In [7]:
classifiers = (
    MultinomialNB(),
    ComplementNB(),
    LogisticRegression(multi_class='auto', solver='lbfgs'),
    RandomForestClassifier(n_estimators=50, min_samples_split=5, random_state=0),
    KNeighborsClassifier(n_neighbors=8, algorithm='auto'),
    MLPClassifier(hidden_layer_sizes=(100, 25), max_iter=500, random_state=0),
    LinearSVC(max_iter=500),
    SVC(gamma='auto', max_iter=500),
)

## TF-IDF

In [10]:
vec_tfidf = TfidfVectorizer(ngram_range=(1, 2))
X_tfidf = vec_tfidf.fit_transform(afrases)

vec_tfidf_tmg = TfidfVectorizer(ngram_range=(1, 2))
X_tfidf_tmg = vec_tfidf_tmg.fit_transform(atweets_mg)

vec_tfidf_tn = TfidfVectorizer(ngram_range=(1, 2))
X_tfidf_tn = vec_tfidf_tn.fit_transform(atitulo_noticias)

In [11]:
print("\nall_datasets")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_tfidf, avalencias))
        matriz_resultados['all']['tfidf'][classifier.__class__.__name__] = acc
    except:
        pass

print("\ntweets_mg")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_tfidf_tmg, aval_tweets_mg))
        matriz_resultados['tweets']['tfidf'][classifier.__class__.__name__] = acc
    except:
        pass

print("\ntitulo_noticias")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_tfidf_tn, aval_titulo_noticias))
        matriz_resultados['titulo_noticias']['tfidf'][classifier.__class__.__name__] = acc
    except:
        pass      


all_datasets
Modelo   : MultinomialNB
Acurácia : 61.22%
--------------------
Modelo   : ComplementNB
Acurácia : 64.71%
--------------------
Modelo   : LogisticRegression
Acurácia : 65.19%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 64.42%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 61.34%
--------------------
Modelo   : MLPClassifier
Acurácia : 63.65%
--------------------
Modelo   : LinearSVC
Acurácia : 66.67%
--------------------
Modelo   : SVC
Acurácia : 59.15%
--------------------

tweets_mg
Modelo   : MultinomialNB
Acurácia : 62.29%
--------------------
Modelo   : ComplementNB
Acurácia : 59.86%
--------------------
Modelo   : LogisticRegression
Acurácia : 64.51%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 63.8%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 61.98%
--------------------
Modelo   : MLPClassifier
Acurácia : 62.18%
--------------------
Modelo   : LinearSVC
Acurácia : 63.9%
----------------

## LSA (usando TF-IDF)

In [12]:
#all_datasets
svd = TruncatedSVD(n_components=70, n_iter=50, random_state=0)
normalizer = MinMaxScaler(copy=False)
lsa = make_pipeline(svd, normalizer)
X_svd = lsa.fit_transform(X_tfidf)

print("\nall_datasets")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_svd, avalencias))
        matriz_resultados['all']['tfidf+lsa'][classifier.__class__.__name__] = acc
    except Exception as e:
        print(e)

#tweets_mg
X_svd = lsa.fit_transform(X_tfidf_tmg)

print("\ntweets_mg")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_svd, aval_tweets_mg))
        matriz_resultados['tweets']['tfidf+lsa'][classifier.__class__.__name__] = acc
    except:
        pass


#titulo_noticias
X_svd = lsa.fit_transform(X_tfidf_tn)

print("\ntitulo_noticias")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_svd, aval_titulo_noticias))
        matriz_resultados['titulo_noticias']['tfidf+lsa'][classifier.__class__.__name__] = acc
    except:
        pass


all_datasets
Modelo   : MultinomialNB
Acurácia : 44.58%
--------------------
Modelo   : ComplementNB
Acurácia : 52.99%
--------------------
Modelo   : LogisticRegression
Acurácia : 61.52%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 61.46%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 55.65%
--------------------
Modelo   : MLPClassifier
Acurácia : 62.52%
--------------------
Modelo   : LinearSVC
Acurácia : 61.99%
--------------------
Modelo   : SVC
Acurácia : 45.29%
--------------------

tweets_mg
Modelo   : MultinomialNB
Acurácia : 45.1%
--------------------
Modelo   : ComplementNB
Acurácia : 55.41%
--------------------
Modelo   : LogisticRegression
Acurácia : 64.91%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 62.49%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 58.85%
--------------------
Modelo   : MLPClassifier
Acurácia : 62.49%
--------------------
Modelo   : LinearSVC
Acurácia : 64.81%
---------------

## LDA (usando TF-IDF)

In [13]:
#all_datasets
lda = LatentDirichletAllocation(n_components=70, max_iter=50, random_state=0, n_jobs=5)
normalizer = MinMaxScaler(copy=False)
lda = make_pipeline(lda, normalizer)
X_lda = lda.fit_transform(X_tfidf)

print("\nall_datasets")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_lda, avalencias))
        matriz_resultados['all']['tfidf+lda'][classifier.__class__.__name__] = acc
    except:
        pass


#tweets_mg
X_lda = lda.fit_transform(X_tfidf_tmg)

print("\ntweets_mg")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_lda, aval_tweets_mg))
        matriz_resultados['tweets']['tfidf+lda'][classifier.__class__.__name__] = acc
    except:
        pass


#titulo_noticias
X_lda = lda.fit_transform(X_tfidf_tn)

print("\ntitulo_noticias")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_lda, aval_titulo_noticias))
        matriz_resultados['titulo_noticias']['tfidf+lda'][classifier.__class__.__name__] = acc
    except:
        pass


all_datasets
Modelo   : MultinomialNB
Acurácia : 46.36%
--------------------
Modelo   : ComplementNB
Acurácia : 43.34%
--------------------
Modelo   : LogisticRegression
Acurácia : 45.77%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 46.36%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 39.14%
--------------------
Modelo   : MLPClassifier
Acurácia : 41.62%
--------------------
Modelo   : LinearSVC
Acurácia : 45.77%
--------------------
Modelo   : SVC
Acurácia : 44.05%
--------------------

tweets_mg
Modelo   : MultinomialNB
Acurácia : 50.46%
--------------------
Modelo   : ComplementNB
Acurácia : 47.83%
--------------------
Modelo   : LogisticRegression
Acurácia : 49.95%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 49.14%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 46.61%
--------------------
Modelo   : MLPClassifier
Acurácia : 52.07%
--------------------
Modelo   : LinearSVC
Acurácia : 50.05%
--------------

## Count

In [14]:
#all_datasets
vec_count = CountVectorizer(ngram_range=(1, 2))
X_count = vec_count.fit_transform(afrases)

print("\nall_datasets")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_count, avalencias))
        matriz_resultados['all']['count'][classifier.__class__.__name__] = acc
    except:
        pass

      
#tweets_mg
vec_count = CountVectorizer(ngram_range=(1, 2))
X_count_tmg = vec_count.fit_transform(atweets_mg)

print("\ntweets_mg")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_count_tmg, aval_tweets_mg))
        matriz_resultados['tweets']['count'][classifier.__class__.__name__] = acc
    except:
        pass
      
      
#titulo_noticias
vec_count = CountVectorizer(ngram_range=(1, 2))
X_count_tn = vec_count.fit_transform(atitulo_noticias)

print("\ntitulo_noticias")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_count_tn, aval_titulo_noticias))
        matriz_resultados['titulo_noticias']['count'][classifier.__class__.__name__] = acc
    except:
        pass


all_datasets
Modelo   : MultinomialNB
Acurácia : 65.19%
--------------------
Modelo   : ComplementNB
Acurácia : 64.18%
--------------------
Modelo   : LogisticRegression
Acurácia : 66.07%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 65.07%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 40.97%
--------------------
Modelo   : MLPClassifier
Acurácia : 64.71%
--------------------
Modelo   : LinearSVC
Acurácia : 64.89%
--------------------
Modelo   : SVC
Acurácia : 49.26%
--------------------

tweets_mg
Modelo   : MultinomialNB
Acurácia : 60.67%
--------------------
Modelo   : ComplementNB
Acurácia : 60.06%
--------------------
Modelo   : LogisticRegression
Acurácia : 63.4%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 64.11%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 48.94%
--------------------
Modelo   : MLPClassifier
Acurácia : 61.38%
--------------------
Modelo   : LinearSVC
Acurácia : 61.78%
---------------

## LSA (usando Count)

In [15]:
#all_datasets
svd = TruncatedSVD(n_components=70, n_iter=50, random_state=0)
normalizer = MinMaxScaler(copy=False)
lda = make_pipeline(svd, normalizer)
X_svd = lda.fit_transform(X_count)

print("\nall_datasets")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_svd, avalencias))
        matriz_resultados['all']['count+lsa'][classifier.__class__.__name__] = acc
    except:
        pass
      

#tweets_mg
X_svd = lda.fit_transform(X_count_tmg)

print("\ntweets_mg")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_svd, aval_tweets_mg))
        matriz_resultados['tweets']['count+lsa'][classifier.__class__.__name__] = acc
    except:
        pass
      
      
#titulos_noticias
X_svd = lda.fit_transform(X_count_tn)

print("\ntitulos_noticias")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_svd, aval_titulo_noticias))
        matriz_resultados['titulo_noticias']['count+lsa'][classifier.__class__.__name__] = acc
    except:
        pass


all_datasets
Modelo   : MultinomialNB
Acurácia : 46.0%
--------------------
Modelo   : ComplementNB
Acurácia : 52.4%
--------------------
Modelo   : LogisticRegression
Acurácia : 60.33%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 60.57%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 56.84%
--------------------
Modelo   : MLPClassifier
Acurácia : 60.45%
--------------------
Modelo   : LinearSVC
Acurácia : 61.1%
--------------------
Modelo   : SVC
Acurácia : 47.19%
--------------------

tweets_mg
Modelo   : MultinomialNB
Acurácia : 52.07%
--------------------
Modelo   : ComplementNB
Acurácia : 56.02%
--------------------
Modelo   : LogisticRegression
Acurácia : 63.3%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 61.38%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 61.07%
--------------------
Modelo   : MLPClassifier
Acurácia : 64.91%
--------------------
Modelo   : LinearSVC
Acurácia : 63.6%
-------------------

## LDA (usando Count)

In [16]:
#all_datasets
lda = LatentDirichletAllocation(n_components=70, max_iter=50, random_state=0, n_jobs=5)
normalizer = MinMaxScaler(copy=False)
lda = make_pipeline(lda, normalizer)
X_lda = lda.fit_transform(X_count)

print("\nall_datasets")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_lda, avalencias))
        matriz_resultados['all']['count+lda'][classifier.__class__.__name__] = acc
    except:
        pass


#tweets_mg
X_lda = lda.fit_transform(X_count_tmg)

print("\ntweets_mg")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_lda, aval_tweets_mg))
        matriz_resultados['tweets']['count+lda'][classifier.__class__.__name__] = acc
    except:
        pass
      
      
#titulo_noticias
X_lda = lda.fit_transform(X_count_tn)

print("\ntitulo_noticias")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_lda, aval_titulo_noticias))
        matriz_resultados['titulo_noticias']['count+lda'][classifier.__class__.__name__] = acc
    except:
        pass


all_datasets
Modelo   : MultinomialNB
Acurácia : 49.67%
--------------------
Modelo   : ComplementNB
Acurácia : 47.9%
--------------------
Modelo   : LogisticRegression
Acurácia : 49.62%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 52.4%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 46.3%
--------------------
Modelo   : MLPClassifier
Acurácia : 48.31%
--------------------
Modelo   : LinearSVC
Acurácia : 49.73%
--------------------
Modelo   : SVC
Acurácia : 44.76%
--------------------

tweets_mg
Modelo   : MultinomialNB
Acurácia : 53.19%
--------------------
Modelo   : ComplementNB
Acurácia : 51.47%
--------------------
Modelo   : LogisticRegression
Acurácia : 53.79%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 53.89%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 49.44%
--------------------
Modelo   : MLPClassifier
Acurácia : 51.87%
--------------------
Modelo   : LinearSVC
Acurácia : 53.79%
-----------------

## Count + TF-IDF + Word2Vec

In [17]:
#all_datasets
# Count
vec_count = CountVectorizer()
X_count = vec_count.fit_transform(afrases)
weights_count = pd.DataFrame(np.round(X_count.toarray().T, 8), index=vec_count.get_feature_names())

# TF-IDF
vec_tfidf = TfidfVectorizer()
X_tfidf = vec_tfidf.fit_transform(afrases)
weights_tfidf = pd.DataFrame(np.round(X_tfidf.toarray().T, 8), index=vec_tfidf.get_feature_names())

# Word2Vec preprocessing
frases_w2v = []
for frase in afrases:
    bigram = []
    p_frase = word_tokenize(frase)
    for m, palavra in enumerate(p_frase):
        next_p = None
        try:
            next_p = p_frase[m+1]
        except:
            pass
        bigram += [f'{palavra}']
#         if next_p:
#             bigram += [f'{palavra} {next_p}']
    frases_w2v += [bigram]

# Word2Vec
model = gensim.models.Word2Vec(
    sentences=frases_w2v,
    sg=1,
    hs=1,
    size=1,
    window=25,
    min_count=1,
    seed=0,
    workers=10)
model.train(frases_w2v, total_examples=len(frases_w2v), epochs=1000)

(37607729, 43013000)

In [18]:
#all_datasets
r_words = {}
for word in vec_count.get_feature_names():
    idx = weights_count.index.get_loc(word)
    w2c_val = .1
    try:
        w2c_val = model.wv[word]
    except:
        pass
    r_words[word] = (weights_tfidf.iloc[idx].values + weights_count.iloc[idx].values) * w2c_val
lwor = list(r_words.keys())
X = np.asarray(list(r_words.values()))
weights = pd.DataFrame(X, index=lwor)
X = X.T

normalizer = Normalizer(copy=False)
X = normalizer.fit_transform(X)

print("\nall_datasets")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X, avalencias))
        matriz_resultados['all']['tfidf+count+w2c'][classifier.__class__.__name__] = acc
    except:
        pass


all_datasets
Modelo   : LogisticRegression
Acurácia : 64.12%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 65.54%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 61.28%
--------------------
Modelo   : MLPClassifier
Acurácia : 61.75%
--------------------
Modelo   : LinearSVC
Acurácia : 64.71%
--------------------
Modelo   : SVC
Acurácia : 55.24%
--------------------


In [19]:
#tweets_mg
# Count
vec_count = CountVectorizer()
X_count = vec_count.fit_transform(atweets_mg)
weights_count = pd.DataFrame(np.round(X_count.toarray().T, 8), index=vec_count.get_feature_names())

# TF-IDF
vec_tfidf = TfidfVectorizer()
X_tfidf = vec_tfidf.fit_transform(atweets_mg)
weights_tfidf = pd.DataFrame(np.round(X_tfidf.toarray().T, 8), index=vec_tfidf.get_feature_names())

# Word2Vec preprocessing
frases_w2v = []
for frase in atweets_mg:
    bigram = []
    p_frase = word_tokenize(frase)
    for m, palavra in enumerate(p_frase):
        next_p = None
        try:
            next_p = p_frase[m+1]
        except:
            pass
        bigram += [f'{palavra}']
#         if next_p:
#             bigram += [f'{palavra} {next_p}']
    frases_w2v += [bigram]

# Word2Vec
model = gensim.models.Word2Vec(
    sentences=frases_w2v,
    sg=1,
    hs=1,
    size=1,
    window=25,
    min_count=1,
    seed=0,
    workers=10)
model.train(frases_w2v, total_examples=len(frases_w2v), epochs=1000)

(21304762, 26219000)

In [20]:
#tweets_mg
r_words = {}
for word in vec_count.get_feature_names():
    idx = weights_count.index.get_loc(word)
    w2c_val = .1
    try:
        w2c_val = model.wv[word]
    except:
        pass
    r_words[word] = (weights_tfidf.iloc[idx].values + weights_count.iloc[idx].values) * w2c_val
lwor = list(r_words.keys())
X = np.asarray(list(r_words.values()))
weights = pd.DataFrame(X, index=lwor)
X = X.T

normalizer = Normalizer(copy=False)
X = normalizer.fit_transform(X)

print("\ntweets_mg")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X, aval_tweets_mg))
        matriz_resultados['tweets']['tfidf+count+w2c'][classifier.__class__.__name__] = acc
    except:
        pass


tweets_mg
Modelo   : LogisticRegression
Acurácia : 64.11%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 63.8%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 64.31%
--------------------
Modelo   : MLPClassifier
Acurácia : 60.06%
--------------------
Modelo   : LinearSVC
Acurácia : 63.4%
--------------------
Modelo   : SVC
Acurácia : 62.99%
--------------------


In [21]:
#titulo_noticias
# Count
vec_count = CountVectorizer()
X_count = vec_count.fit_transform(atitulo_noticias)
weights_count = pd.DataFrame(np.round(X_count.toarray().T, 8), index=vec_count.get_feature_names())

# TF-IDF
vec_tfidf = TfidfVectorizer()
X_tfidf = vec_tfidf.fit_transform(atitulo_noticias)
weights_tfidf = pd.DataFrame(np.round(X_tfidf.toarray().T, 8), index=vec_tfidf.get_feature_names())

# Word2Vec preprocessing
frases_w2v = []
for frase in atitulo_noticias:
    bigram = []
    p_frase = word_tokenize(frase)
    for m, palavra in enumerate(p_frase):
        next_p = None
        try:
            next_p = p_frase[m+1]
        except:
            pass
        bigram += [f'{palavra}']
#         if next_p:
#             bigram += [f'{palavra} {next_p}']
    frases_w2v += [bigram]

# Word2Vec
model = gensim.models.Word2Vec(
    sentences=frases_w2v,
    sg=1,
    hs=1,
    size=1,
    window=25,
    min_count=1,
    seed=0,
    workers=10)
model.train(frases_w2v, total_examples=len(frases_w2v), epochs=1000)

(13988216, 16794000)

In [22]:
#titulo_noticias
r_words = {}
for word in vec_count.get_feature_names():
    idx = weights_count.index.get_loc(word)
    w2c_val = .1
    try:
        w2c_val = model.wjv[word]
    except:
        pass
    r_words[word] = (weights_tfidf.iloc[idx].values + weights_count.iloc[idx].values) * w2c_val
lwor = list(r_words.keys())
X = np.asarray(list(r_words.values()))
weights = pd.DataFrame(X, index=lwor)
X = X.T

normalizer = Normalizer(copy=False)
X = normalizer.fit_transform(X)

print("\ntitulo_noticias")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X, aval_titulo_noticias))
        matriz_resultados['titulo_noticias']['tfidf+count+w2c'][classifier.__class__.__name__] = acc
    except:
        pass


titulo_noticias
Modelo   : MultinomialNB
Acurácia : 62.77%
--------------------
Modelo   : ComplementNB
Acurácia : 64.05%
--------------------
Modelo   : LogisticRegression
Acurácia : 66.9%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 65.05%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 62.2%
--------------------
Modelo   : MLPClassifier
Acurácia : 65.76%
--------------------
Modelo   : LinearSVC
Acurácia : 67.19%
--------------------
Modelo   : SVC
Acurácia : 48.79%
--------------------


### Resultado dos Classificadores

#### Resultado dos Classificadores para todas as frases

In [23]:
df = pd.DataFrame.from_dict(matriz_resultados['all'])
df.style.apply(highlight_max).format({
    'tfidf': '{:,.2f}%'.format,
    'tfidf+lsa': '{:,.2f}%'.format,
    'tfidf+lda': '{:,.2f}%'.format,
    'count': '{:,.2f}%'.format,
    'count+lsa': '{:,.2f}%'.format,
    'count+lda': '{:,.2f}%'.format,
    'tfidf+count+w2c': '{:,.2f}%'.format,
})

,tfidf,tfidf+lsa,tfidf+lda,count,count+lsa,count+lda,tfidf+count+w2c
ComplementNB,64.71%,52.99%,43.34%,64.18%,52.40%,47.90%,0.00%
KNeighborsClassifier,61.34%,55.65%,39.14%,40.97%,56.84%,46.30%,61.28%
LinearSVC,66.67%,61.99%,45.77%,64.89%,61.10%,49.73%,64.71%
LogisticRegression,65.19%,61.52%,45.77%,66.07%,60.33%,49.62%,64.12%
MLPClassifier,63.65%,62.52%,41.62%,64.71%,60.45%,48.31%,61.75%
MultinomialNB,61.22%,44.58%,46.36%,65.19%,46.00%,49.67%,0.00%
RandomForestClassifier,64.42%,61.46%,46.36%,65.07%,60.57%,52.40%,65.54%
SVC,59.15%,45.29%,44.05%,49.26%,47.19%,44.76%,55.24%


#### Resultado dos Classificadores para os Tweets MG

In [24]:
df = pd.DataFrame.from_dict(matriz_resultados['tweets'])
df.style.apply(highlight_max).format({
    'tfidf': '{:,.2f}%'.format,
    'tfidf+lsa': '{:,.2f}%'.format,
    'tfidf+lda': '{:,.2f}%'.format,
    'count': '{:,.2f}%'.format,
    'count+lsa': '{:,.2f}%'.format,
    'count+lda': '{:,.2f}%'.format,
    'tfidf+count+w2c': '{:,.2f}%'.format,
})

,tfidf,tfidf+lsa,tfidf+lda,count,count+lsa,count+lda,tfidf+count+w2c
ComplementNB,59.86%,55.41%,47.83%,60.06%,56.02%,51.47%,0.00%
KNeighborsClassifier,61.98%,58.85%,46.61%,48.94%,61.07%,49.44%,64.31%
LinearSVC,63.90%,64.81%,50.05%,61.78%,63.60%,53.79%,63.40%
LogisticRegression,64.51%,64.91%,49.95%,63.40%,63.30%,53.79%,64.11%
MLPClassifier,62.18%,62.49%,52.07%,61.38%,64.91%,51.87%,60.06%
MultinomialNB,62.29%,45.10%,50.46%,60.67%,52.07%,53.19%,0.00%
RandomForestClassifier,63.80%,62.49%,49.14%,64.11%,61.38%,53.89%,63.80%
SVC,61.58%,45.60%,46.41%,43.98%,53.89%,48.84%,62.99%


#### Resultados dos Classificadores para os Títulos de Notícias

In [25]:
df = pd.DataFrame.from_dict(matriz_resultados['titulo_noticias'])
df.style.apply(highlight_max).format({
    'tfidf': '{:,.2f}%'.format,
    'tfidf+lsa': '{:,.2f}%'.format,
    'tfidf+lda': '{:,.2f}%'.format,
    'count': '{:,.2f}%'.format,
    'count+lsa': '{:,.2f}%'.format,
    'count+lda': '{:,.2f}%'.format,
    'tfidf+count+w2c': '{:,.2f}%'.format,
})

,tfidf,tfidf+lsa,tfidf+lda,count,count+lsa,count+lda,tfidf+count+w2c
ComplementNB,64.05%,57.77%,38.52%,62.91%,50.50%,42.80%,64.05%
KNeighborsClassifier,63.48%,56.49%,41.51%,42.65%,54.64%,41.51%,62.20%
LinearSVC,68.05%,60.49%,42.37%,68.62%,58.92%,42.94%,67.19%
LogisticRegression,65.76%,59.63%,42.51%,69.04%,57.77%,43.51%,66.90%
MLPClassifier,67.05%,62.91%,43.37%,67.05%,55.78%,40.09%,65.76%
MultinomialNB,62.34%,43.08%,42.65%,67.76%,43.08%,43.94%,62.77%
RandomForestClassifier,64.91%,63.48%,42.80%,63.77%,60.06%,44.22%,65.05%
SVC,49.22%,44.37%,42.08%,46.65%,47.50%,43.94%,48.79%


### Modelo escolhido e salvo

In [8]:
vec_tfidf = TfidfVectorizer(ngram_range=(1, 2))
X_tfidf = vec_tfidf.fit_transform(afrases)

svd = TruncatedSVD(n_components=70, n_iter=50, random_state=0)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)
X_svd = lsa.fit_transform(X_tfidf)

svm = LinearSVC(max_iter=1200)
model = CalibratedClassifierCV(svm) 
model.fit(X_svd, avalencias)

accuracy = np.round(model.score(X_svd, avalencias) * 100, 2)
print(f'Modelo   : {model.__class__.__name__}')
print(f'Acurácia : {accuracy}%')


filename = 'tfidf_valence.sav'
joblib.dump(vec_tfidf, filename)

filename = 'lsa_valence.sav'
joblib.dump(lsa, filename)

filename = 'model_valence.sav'
joblib.dump(model, filename)

Modelo   : CalibratedClassifierCV
Acurácia : 62.31%


['model_valence.sav']